### 서울시 데이터 통합

In [10]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

##### 1. CCTV 데이터

In [11]:
# 1) 데이터 파일 읽기
filename = 'data/서울시 자치구 년도별 CCTV 설치 현황_221231기준.csv'
columns = '구분,총계,2013년 이전 설치된 CCTV,2013년,2014년,2015년,2016년,2017년,2018년,2019년,2020년,2021년,2022년'.split(',')
cctv = pd.read_csv(filename, encoding='euc=kr', thousands=',')
cctv = cctv[columns]
cctv.head()

,구분,총계,2013년 이전 설치된 CCTV,2013년,2014년,2015년,2016년,2017년,2018년,2019년,2020년,2021년,2022년
0,계,91341,2804,3097,3981,6576,8850,11566,11024,13151,11912,9144,9236
1,종로구,1980,36,540,107,161,131,158,152,69,250,85,291
2,중 구,2584,130,87,77,236,240,372,386,155,361,403,137
3,용산구,2847,44,50,68,83,295,491,115,322,623,422,334
4,성동구,4047,58,99,110,366,279,945,459,647,485,367,232


In [12]:
# 2) 결측치 확인
cctv.isna().sum().sum()

0

In [13]:
# 3) 최근 증가율
cctv.loc[:, '2020년':'2022년'].sum(axis=1).head()

0    30292
1      626
2      901
3     1379
4     1084
dtype: int64

In [14]:
cctv['최근증가율'] = cctv.iloc[:, 10:].sum(axis=1) / cctv.iloc[:, 2:10].sum(axis=1)
cctv['최근증가율'] = (cctv['최근증가율'] * 100).round(2)
cctv.head()

,구분,총계,2013년 이전 설치된 CCTV,2013년,2014년,2015년,2016년,2017년,2018년,2019년,2020년,2021년,2022년,최근증가율
0,계,91341,2804,3097,3981,6576,8850,11566,11024,13151,11912,9144,9236,49.62
1,종로구,1980,36,540,107,161,131,158,152,69,250,85,291,46.23
2,중 구,2584,130,87,77,236,240,372,386,155,361,403,137,53.54
3,용산구,2847,44,50,68,83,295,491,115,322,623,422,334,93.94
4,성동구,4047,58,99,110,366,279,945,459,647,485,367,232,36.58


In [15]:
# 4) Selection and rename
cctv = cctv[['구분', '총계', '최근증가율']]
cctv.head()

,구분,총계,최근증가율
0,계,91341,49.62
1,종로구,1980,46.23
2,중 구,2584,53.54
3,용산구,2847,93.94
4,성동구,4047,36.58


In [16]:
cctv.rename(columns={'구분':'구별', '총계':'CCTV댓수'}, inplace=True)
cctv.head()

,구별,CCTV댓수,최근증가율
0,계,91341,49.62
1,종로구,1980,46.23
2,중 구,2584,53.54
3,용산구,2847,93.94
4,성동구,4047,36.58


In [17]:
# 5) Filtering - 계 데이터 삭제
cctv.drop([0], inplace=True)
cctv.head()

,구별,CCTV댓수,최근증가율
1,종로구,1980,46.23
2,중 구,2584,53.54
3,용산구,2847,93.94
4,성동구,4047,36.58
5,광진구,3480,46.53


In [18]:
# 6) 구 이름에서 공백 지우기
cctv.구별 = cctv.구별.str.replace(' ', '')
cctv.head(3)

,구별,CCTV댓수,최근증가율
1,종로구,1980,46.23
2,중구,2584,53.54
3,용산구,2847,93.94


In [19]:
# 7) cctv.csv 파일에 저장
cctv.to_csv('data/cctv.csv', index=False)
df = pd.read_csv('data/cctv.csv')
df.head()

,구별,CCTV댓수,최근증가율
0,종로구,1980,46.23
1,중구,2584,53.54
2,용산구,2847,93.94
3,성동구,4047,36.58
4,광진구,3480,46.53


##### 2. 인구 데이터

In [20]:
# 1) 파일 읽기
pop = pd.read_csv('data/서울시인구.txt', sep='\t')
pop.head()

,기간,자치구,세대,인구,인구.1,인구.2,인구.3,인구.4,인구.5,인구.6,인구.7,인구.8,세대당인구,65세이상고령자
0,기간,자치구,세대,합계,합계,합계,한국인,한국인,한국인,등록외국인,등록외국인,등록외국인,세대당인구,65세이상고령자
1,기간,자치구,세대,계,남자,여자,계,남자,여자,계,남자,여자,세대당인구,65세이상고령자
2,2021,합계,"4,426,007","9,736,027","4,721,977","5,014,050","9,509,458","4,618,040","4,891,418","226,569","103,937","122,632",2.15,"1,605,416"
3,2021,종로구,"73,494","153,789","74,186","79,603","144,683","70,183","74,500","9,106","4,003","5,103",1.97,"27,818"
4,2021,중구,"63,519","131,787","64,083","67,704","122,499","59,630","62,869","9,288","4,453","4,835",1.93,"24,392"


In [21]:
pop = pd.read_csv('data/서울시인구.txt', sep='\t', skiprows=2)
pop.head()

,기간,자치구,세대,계,남자,여자,계.1,남자.1,여자.1,계.2,남자.2,여자.2,세대당인구,65세이상고령자
0,2021,합계,"4,426,007","9,736,027","4,721,977","5,014,050","9,509,458","4,618,040","4,891,418","226,569","103,937","122,632",2.15,"1,605,416"
1,2021,종로구,"73,494","153,789","74,186","79,603","144,683","70,183","74,500","9,106","4,003","5,103",1.97,"27,818"
2,2021,중구,"63,519","131,787","64,083","67,704","122,499","59,630","62,869","9,288","4,453","4,835",1.93,"24,392"
3,2021,용산구,"111,036","237,285","115,085","122,200","222,953","107,210","115,743","14,332","7,875","6,457",2.01,"39,070"
4,2021,성동구,"134,233","292,672","142,259","150,413","285,990","139,380","146,610","6,682","2,879","3,803",2.13,"46,380"


In [22]:
# 2) 결측치 확인
pop.isna().sum().sum()

0

In [23]:
# 3) Selection and rename
pop = pop[['자치구','계','계.1','계.2','65세이상고령자']]
pop.head(3)

,자치구,계,계.1,계.2,65세이상고령자
0,합계,"9,736,027","9,509,458","226,569","1,605,416"
1,종로구,"153,789","144,683","9,106","27,818"
2,중구,"131,787","122,499","9,288","24,392"


In [24]:
pop.columns = ['구별','인구수','내국인','외국인','고령자']
pop.head(3)

,구별,인구수,내국인,외국인,고령자
0,합계,"9,736,027","9,509,458","226,569","1,605,416"
1,종로구,"153,789","144,683","9,106","27,818"
2,중구,"131,787","122,499","9,288","24,392"


In [25]:
# 4) 합계 데이터 삭제
pop.drop([0], inplace=True)
pop.head(3)

,구별,인구수,내국인,외국인,고령자
1,종로구,"153,789","144,683","9,106","27,818"
2,중구,"131,787","122,499","9,288","24,392"
3,용산구,"237,285","222,953","14,332","39,070"


In [26]:
# 5) 천단위 구분기호 없애고 정수로 변환하기
for column in pop.columns[1:]:
    pop[column] = pop[column].str.replace(',','').astype(int)
pop.head(3)

,구별,인구수,내국인,외국인,고령자
1,종로구,153789,144683,9106,27818
2,중구,131787,122499,9288,24392
3,용산구,237285,222953,14332,39070


In [27]:
# 6) 외국인 비율, 고령자 비율 컬럼 만들기
pop['외국인비율'] = (pop.외국인 / pop.인구수 * 100).round(2)
pop['고령자비율'] = (pop.고령자 / pop.인구수 * 100).round(2)
pop.head(3)

,구별,인구수,내국인,외국인,고령자,외국인비율,고령자비율
1,종로구,153789,144683,9106,27818,5.92,18.09
2,중구,131787,122499,9288,24392,7.05,18.51
3,용산구,237285,222953,14332,39070,6.04,16.47


In [28]:
# 7) pop.csv에 저장하기
pop.to_csv('data/pop.csv', index=False)
df = pd.read_csv('data/pop.csv')
df.head(3)

,구별,인구수,내국인,외국인,고령자,외국인비율,고령자비율
0,종로구,153789,144683,9106,27818,5.92,18.09
1,중구,131787,122499,9288,24392,7.05,18.51
2,용산구,237285,222953,14332,39070,6.04,16.47
